<a href="https://colab.research.google.com/github/King1oo1/AI_Art_Therapy_Project/blob/main/Week2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# AI Art Therapy Project - Fixed Version
# This version should render properly on GitHub

# Step 1: Installation and Imports
import torch
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter, ImageDraw, ImageEnhance
import cv2
from diffusers import StableDiffusionPipeline
import random
import json
from datetime import datetime
from typing import Dict, List, Tuple
import math

print("✅ AI Art Therapy Project Initialized!")

# Step 2: Emotional Art Generator
class EmotionalStableDiffusionGenerator:
    """Advanced emotional art generator using Stable Diffusion"""

    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.setup_models()
        self.setup_emotional_prompts()

    def setup_models(self):
        """Load Stable Diffusion models"""
        print("🔄 Loading AI models...")

        try:
            self.pipe = StableDiffusionPipeline.from_pretrained(
                "runwayml/stable-diffusion-v1-5",
                torch_dtype=torch.float16,
                safety_checker=None,
                requires_safety_checker=False
            ).to(self.device)
            print("✅ Model loaded successfully!")
        except Exception as e:
            print(f"❌ Model loading failed: {e}")
            self.pipe = None

    def setup_emotional_prompts(self):
        """Emotional prompt engineering"""
        self.emotion_prompts = {
            "joy": {
                "descriptors": ["vibrant", "colorful", "radiant", "glowing", "celebratory"],
                "themes": ["celebration", "sunrise", "festival", "blossoming flowers"]
            },
            "sadness": {
                "descriptors": ["gentle", "soft", "melancholic", "dreamy", "reflective"],
                "themes": ["rainy day", "empty space", "fading light", "lonely landscape"]
            },
            "anger": {
                "descriptors": ["intense", "dynamic", "powerful", "fiery", "explosive"],
                "themes": ["storm", "fire", "shattered glass", "chaos"]
            },
            "fear": {
                "descriptors": ["mysterious", "atmospheric", "surreal", "haunting"],
                "themes": ["fog", "shadows", "unknown territory", "surreal dream"]
            }
        }

    def keyword_emotion_analysis(self, text):
        """Simple emotion analysis based on keywords"""
        text_lower = text.lower()

        emotion_keywords = {
            "joy": ["happy", "joy", "excited", "great", "wonderful"],
            "sadness": ["sad", "depressed", "unhappy", "hurt", "grief"],
            "anger": ["angry", "mad", "frustrated", "furious", "rage"],
            "fear": ["scared", "afraid", "anxious", "worried", "terrified"]
        }

        scores = {emotion: 0 for emotion in emotion_keywords}

        for emotion, keywords in emotion_keywords.items():
            for keyword in keywords:
                if keyword in text_lower:
                    scores[emotion] += 1

        if sum(scores.values()) == 0:
            return {"primary": "neutral", "intensity": 0.5}

        primary_emotion = max(scores.items(), key=lambda x: x[1])
        return {"primary": primary_emotion[0], "intensity": primary_emotion[1] / 5}

    def create_emotional_prompt(self, emotion_data, style):
        """Create emotional art prompt"""
        emotion = emotion_data["primary"]
        emotion_info = self.emotion_prompts.get(emotion, self.emotion_prompts["joy"])

        descriptor = random.choice(emotion_info["descriptors"])
        theme = random.choice(emotion_info["themes"])

        prompt = f"masterpiece, best quality, {descriptor} {emotion}, {theme}, {style}, emotional art, atmospheric"
        negative_prompt = "ugly, deformed, noisy, blurry, low quality, human faces, text, watermark"

        return prompt, negative_prompt

    def generate_artwork(self, user_text, style="digital art"):
        """Generate emotionally responsive artwork"""
        emotion_data = self.keyword_emotion_analysis(user_text)
        prompt, negative_prompt = self.create_emotional_prompt(emotion_data, style)

        print(f"🎨 Generating art for emotion: {emotion_data['primary']}")

        if self.pipe is None:
            return self.generate_fallback_art(emotion_data)

        try:
            result = self.pipe(
                prompt=prompt,
                negative_prompt=negative_prompt,
                guidance_scale=7.5,
                num_inference_steps=20,
                width=512,
                height=512,
                generator=torch.Generator(device=self.device).manual_seed(random.randint(0, 2**32))
            )

            return {
                "image": result.images[0],
                "emotion_data": emotion_data,
                "prompt": prompt
            }

        except Exception as e:
            print(f"❌ Generation failed: {e}")
            return self.generate_fallback_art(emotion_data)

    def generate_fallback_art(self, emotion_data):
        """Generate fallback mathematical art"""
        print("🎨 Creating mathematical fallback art...")

        emotion_colors = {
            "joy": [(255, 223, 0), (255, 105, 180), (50, 205, 50)],
            "sadness": [(100, 149, 237), (70, 130, 180), (176, 196, 222)],
            "anger": [(220, 20, 60), (178, 34, 34), (139, 0, 0)],
            "fear": [(75, 0, 130), (106, 90, 205), (72, 61, 139)],
            "neutral": [(128, 128, 128), (169, 169, 169), (105, 105, 105)]
        }

        colors = emotion_colors.get(emotion_data["primary"], emotion_colors["neutral"])
        size = 512

        img = Image.new('RGB', (size, size), colors[0])
        draw = ImageDraw.Draw(img)

        # Create simple pattern based on emotion
        for i in range(size):
            wave_height = int(50 * math.sin(i * 0.05))
            color = colors[i % len(colors)]
            draw.line([(i, size//2 - wave_height), (i, size//2 + wave_height)], fill=color, width=2)

        return {
            "image": img,
            "emotion_data": emotion_data,
            "prompt": f"Mathematical art expressing {emotion_data['primary']}"
        }

# Initialize generator
print("🚀 Initializing Emotional Art Generator...")
try:
    art_generator = EmotionalStableDiffusionGenerator()
    print("✅ Generator ready!")
except Exception as e:
    print(f"❌ Initialization failed: {e}")

# Step 3: Create Gradio Interface
def create_art_interface(text, style):
    """Create interface for art generation"""
    if not text.strip():
        return None, "Please enter your feelings or thoughts"

    try:
        result = art_generator.generate_artwork(text, style)
        return result["image"], f"Emotion: {result['emotion_data']['primary']}\nPrompt: {result['prompt']}"
    except Exception as e:
        return None, f"Error: {str(e)}"

# Step 4: Demo and Testing
if __name__ == "__main__":
    # Test the generator
    print("🧪 Testing emotional art generation...")

    test_texts = [
        "I feel incredibly joyful and excited!",
        "Feeling deeply sad and melancholic today",
        "I'm furious and angry about what happened",
        "Anxious and scared about the future"
    ]

    # Create a simple demo
    for i, text in enumerate(test_texts[:2]):
        print(f"\nTest {i+1}: {text}")
        try:
            result = art_generator.generate_artwork(text)
            print(f"✅ Generated art for: {result['emotion_data']['primary']}")
        except Exception as e:
            print(f"❌ Failed: {e}")

    print("🎉 AI Art Therapy Project is ready!")

✅ AI Art Therapy Project Initialized!
🚀 Initializing Emotional Art Generator...
🔄 Loading AI models...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

✅ Model loaded successfully!
✅ Generator ready!
🧪 Testing emotional art generation...

Test 1: I feel incredibly joyful and excited!
🎨 Generating art for emotion: joy


  0%|          | 0/20 [00:00<?, ?it/s]

✅ Generated art for: joy

Test 2: Feeling deeply sad and melancholic today
🎨 Generating art for emotion: sadness


  0%|          | 0/20 [00:00<?, ?it/s]

✅ Generated art for: sadness
🎉 AI Art Therapy Project is ready!
